## Link para o arquivo de batimetria: https://www.bodc.ac.uk/data/open_download/gebco/gebco_2024_sub_ice_topo/zip/

In [2]:
import xarray as xr
import pygmt
import matplotlib.pyplot as plt

# Carrega o dataset batimétrico GEBCO
bat = xr.open_dataset("GEBCO_2023_sub_ice_topo.nc")

# Seleciona a região de interesse entre as latitudes -70 a 30 e longitudes -70 a -10
bat = bat.sel(lat=slice(-70, 30), lon=slice(-70, -10))

# Reduz a resolução dos dados com média em blocos de 10 graus (coarsen)
bat_coarse = bat.coarsen(lat=10, lon=10, boundary='trim').mean()

# Seleciona a variável 'elevation' para criar a grade e ajusta valores positivos (terra) dividindo por 10
grid = bat_coarse.elevation
grid = xr.where(grid > 0, grid / 10, grid)


In [ ]:
fig = pygmt.Figure()

# Configurações gerais de mapa e fonte
with pygmt.config(FORMAT_GEO_MAP="ddd.xxF", MAP_FRAME_TYPE="fancy", FONT='22p'):
    # Cria um gráfico 3D da grade batimétrica
    fig.grdview(
        grid=grid,  # Grade de elevação criada anteriormente
        region=[-69.9, -10.1, -69.9, 29.9, -8248, 5493],  # Define a região e os limites de elevação
        perspective=[160, 30],  # Define a perspectiva 3D
        frame=["xa", "yaf", "WSnE"],  # Define o frame do mapa com rótulos nos eixos
        projection="Q0/0/100c",  # Projeção em perspectiva 3D
        zsize="50c",  # Altura da projeção em relação ao plano
        surftype="s",  # Tipo de superfície a ser exibida
        interpolation='c',  # Interpolação cúbica dos dados
        plane="-5500+ggrey",  # Define o plano de fundo da superfície
        cmap="geo",  # Mapa de cores baseado na elevação geográfica
    )
    
    # Adiciona a rosa dos ventos na parte superior (mapa) com um leve deslocamento
    fig.basemap(
        perspective=True,
        rose="JBR+w5c+l+o0c/0.5c"
    )

# Configurações para a barra de cores
with pygmt.config(FONT='20p'):
    fig.colorbar(
        perspective=True,
        frame=["a1000", "x+lElevation", "y+lm"],  # Rótulos da barra de cores e intervalo
        position="JRM+o4c+w20/0.5c",  # Posição e tamanho da barra de cores
        truncate=[-5000, 0]  # Limites da barra de cores
    )

# Exibe o gráfico
fig.show()

# Salva o gráfico como uma imagem PNG
fig.savefig('Batimetria.png')
